<a href="https://colab.research.google.com/github/srikanthrc/running-to-stand-still/blob/master/_notebooks/2020-03-21-covid19_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COVID-19 Tracking / Worldwide
> Tracking coronavirus total cases, deaths and new cases by country.  Separately, a view of US by state.

- author: Srikanth Chinmay (inspired by Pratap Vardhan @ http://covid19dashboards.com)
- categories: [covid-19]
- image: images/covid-overview.png
- permalink: /covid-overview/
- toc: false
- comments: false


In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [0]:
#hide
import numpy as np
import pandas as pd
from datetime import datetime
from jinja2 import Template
from IPython.display import HTML

In [0]:
#hide

# FETCH
import getpass
base_url = 'https://raw.githubusercontent.com/srikanthrc/covid-19/master/'
base_url = '' if (getpass.getuser() == 'Pratap Vardhan') else base_url
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
        # f'csse_covid_19_time_series/time_series_19-covid-{name}.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [0]:
#hide
COL_REGION = 'Country/Region'

df = get_frame('confirmed')

LAST_DATE_I, dt_cols = get_dates(df)
dt_today = dt_cols[LAST_DATE_I]
dt_ago = dt_cols[LAST_DATE_I-1]

# dft_: timeseries, dfc_: today country agg
# Confirmed, Deaths
dft_cases = df
dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_ago].sum()

dft_deaths = get_frame('deaths')
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_ago].sum()

#dft_recovered = get_frame('recovered')
#dfc_recovered = dft_recovered.groupby(COL_REGION)[dt_today].sum()
#dfp_recovered = dft_recovered.groupby(COL_REGION)[dt_ago].sum()


In [5]:
#hide
df_table = (pd.DataFrame(dict(
    Cases=dfc_cases, Deaths=dfc_deaths, 
    PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

,Country/Region,Cases,Deaths,PCases,PDeaths,Cases (+),Deaths (+),Fatality Rate,Continent
0,US,461437,16478,429052,14695,32385,1783,3.6,North America
1,Spain,153222,15447,148220,14792,5002,655,10.1,Europe
2,Italy,143626,18279,139422,17669,4204,610,12.7,Europe
3,France,118781,12228,113959,10887,4822,1341,10.3,Europe
4,Germany,118181,2607,113296,2349,4885,258,2.2,Europe
5,China,82883,3339,82809,3337,74,2,4.0,Asia
6,Iran,66220,4110,64586,3993,1634,117,6.2,Asia
7,United Kingdom,65872,7993,61474,7111,4398,882,12.1,Europe
8,Turkey,42282,908,38226,812,4056,96,2.1,Asia
9,Belgium,24983,2523,23403,2240,1580,283,10.1,Europe


In [6]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'Cases': 1595350,
 'Cases (+)': 84246,
 'China Cases': 82883,
 'China Cases (+)': 74,
 'China Deaths': 3339,
 'China Deaths (+)': 2,
 'Deaths': 95455,
 'Deaths (+)': 7117,
 'EU Cases': 780027,
 'EU Cases (+)': 33895,
 'EU Deaths': 65852,
 'EU Deaths (+)': 4553,
 'US Cases': 461437,
 'US Cases (+)': 32385,
 'US Deaths': 16478,
 'US Deaths (+)': 1783,
 'since': Timestamp('2020-04-08 00:00:00'),
 'updated': Timestamp('2020-04-09 00:00:00')}

In [0]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
#dft_ct_new_cases.head()
#dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-10]:dt_cols[LAST_DATE_I]]
#df_table.head(20)

In [8]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table.head(20),  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

#####Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Sources: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub](https://github.com/CSSEGISandData/COVID-19). [covidtracking.com](https://covidtracking.com/)